In [2]:
from models.yolo import Detect
from utils.general import scale_coords, non_max_suppression, xyxy2xywh, xyxy2xywhn
import coremltools
from PIL import Image
import torch
import numpy as np
import random
import cv2
import os
import shutil
import PIL.Image

In [3]:
# Parameters and Global variables
MODE = 'debug'
SAVE_IMG = True
VIEW_IMG = False
SAVE_TXT = True
CAT_NAMES = ['Screw', 'unknown']

# Anchor box can be checked in pytorch model
ANCHORS = ([2.375,3.375, 5.5,5, 4.75,11.75], 
           [6,4.25, 5.375,9.5, 11.25,8.5625], 
           [4.375,9.40625, 9.46875,8.25, 7.43750,16.93750],
           [6.81250,9.60938, 11.54688,5.93750, 14.45312,12.37500])
# stide can be check in pytorch model
stride = [8, 16, 32, 64]
# target size of input image (width, height)
IMG_SIZE = (2304, 3072)
# confidence threshold
conf_thres = .2


# Params and global variables that should be kept as it is
COLORS = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(CAT_NAMES))]
PATH = "./"
nc = len(CAT_NAMES)
nl = len(ANCHORS)
na = len(ANCHORS[0]) // 2
no = nc + 5
# initiate grid and anchor_grid
grid = [torch.zeros(1)] * nl
a = torch.tensor(ANCHORS).float().view(nl, -1, 2)
anchor_grid = [torch.zeros(1)] * nl

COREML_MODEL = "/Users/zhenyu/Box/MLProject:IphoneAOI/weights/yolov5l6_3072*2304_20211116/weights/best.mlmodel"
IMAGE_FOLDER = "/Users/zhenyu/Desktop/val/"
OUT_FOLDER = "/Users/zhenyu/Desktop/test/"

In [4]:
def make_grid(nx=20, ny=20, i=0):
    yv, xv = torch.meshgrid([torch.arange(ny), torch.arange(nx)])
    grid = torch.stack((xv, yv), 2).view((1, 1, ny, nx, 2)).float()
    anchor_grid = (a[i] * stride[i]).view((1, na, 1, 1, 2)).expand(1, na, ny, nx, 2).float()
    return grid, anchor_grid

def resize_image(source_image):
    background = PIL.Image.new('RGB', IMG_SIZE, "black")
    source_image.thumbnail(IMG_SIZE)
    (w, h) = source_image.size
    background.paste(source_image, (int((IMG_SIZE[0] - w) / 2), int((IMG_SIZE[1] - h) / 2 )))
    return background

In [5]:
def eval(file_name):   
    source = PIL.Image.open(os.path.join(IMAGE_FOLDER, file_name))
    resized = resize_image(source.copy())

    predictions = model.predict({'image': resized})

    z = []  # inference output
    x = []
    for head in ['var_1763', 'var_1778', 'var_1793', 'var_1808']:
        x.append(torch.Tensor(predictions[head]))

    for i in range(nl):
        bs, _, ny, nx, _ = x[i].shape

        # if grid[i].shape[2:4] != x[i].shape[2:4]:
        grid[i], anchor_grid = make_grid(nx, ny, i)

        y = x[i].sigmoid()
        y[..., 0:2] = (y[..., 0:2] * 2. - 0.5 + grid[i]) * stride[i]  # xy
        y[..., 2:4] = (y[..., 2:4] * 2) ** 2 * anchor_grid  # wh
        z.append(y.view(bs, -1, no))
    
    pred = torch.cat(z, 1)

    pred = non_max_suppression(pred, conf_thres, .3, classes=None, agnostic=False)

    # Process detections
    for i, det in enumerate(pred):  # detections per image
        p, s = "./", ""
        label=[]

        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det = det[((det[:, 0]-det[:, 2])*(det[:, 1]-det[:, 3])) > 80]
            for *xyxy, conf, cls in det:
                if SAVE_TXT:
                    xywh = xyxy2xywhn(torch.tensor(xyxy).view(1, 4), w=IMG_SIZE[0], h=IMG_SIZE[1]).view(-1).tolist()
                    label.append(('%g ' * 5 + '\n') % (cls, *xywh))
                if SAVE_IMG:
                    draw = PIL.ImageDraw.Draw(resized)
                    draw.rectangle(np.array(torch.tensor(xyxy).view(2,2)), outline='red')
        if SAVE_TXT:
            with open(os.path.join(OUT_FOLDER, '{}.txt'.format(file_name[:-4])), 'a') as f:
                for line in label:
                    f.write(line)
        if SAVE_IMG:
            resized.save(os.path.join(OUT_FOLDER, '{}.jpg'.format(file_name[:-4])))

In [6]:
model = coremltools.models.model.MLModel(COREML_MODEL)

In [7]:
eval('1_4032.jpg')

In [26]:
def get_anchor():
    from models.experimental import attempt_load  # scoped to avoid circular import
    w = '/Users/zhenyu/Box/MLProject:IphoneAOI/weights/yolov5l6_3072*2304_20211116/weights/best.pt'
    model_pt = torch.jit.load(w) if 'torchscript' in w else attempt_load(w)
    print('Anchors are:\n {}'.format(model_pt.model[-1].anchors))
    print(model_pt.model[-1].grid[0].shape)
    print(model_pt.model[-1].grid[1].shape)
    print(model_pt.model[-1].grid[2].shape)
    print(model_pt.model[-1].grid[3].shape)

In [27]:
get_anchor()

Fusing layers... 
Model Summary: 476 layers, 76126356 parameters, 0 gradients, 110.0 GFLOPs


Anchors are:
 tensor([[[ 2.37500,  3.37500],
         [ 5.50000,  5.00000],
         [ 4.75000, 11.75000]],

        [[ 6.00000,  4.25000],
         [ 5.37500,  9.50000],
         [11.25000,  8.56250]],

        [[ 4.37500,  9.40625],
         [ 9.46875,  8.25000],
         [ 7.43750, 16.93750]],

        [[ 6.81250,  9.60938],
         [11.54688,  5.93750],
         [14.45312, 12.37500]]])
torch.Size([1, 3, 392, 296, 2])
torch.Size([1, 3, 196, 148, 2])
torch.Size([1, 3, 98, 74, 2])
torch.Size([1, 3, 49, 37, 2])
